In [ ]:
import pandas as pd
import os
import glob

os.listdir()

In [ ]:
os.chdir("../")

## Extracts information like actual age, estimated bone age, gender and radiologist initials from the text file.

In [ ]:
import re

def calculate_total_months(duration):
    total_months = 0

    # Extract years and months from the duration string
    years = re.findall(r'(\d+)\s*y', duration, re.IGNORECASE)
    months = re.findall(r'(\d+)\s*m', duration, re.IGNORECASE)

    # Add the years and months to the total
    if years:
        total_months += int(years[0]) * 12
    if months:
        total_months += int(months[0])

    # If no years or months are found, check for a duration in months
    if not years and not months:
        months_only = re.findall(r'(\d+)\s*months?', duration, re.IGNORECASE)
        if months_only:
            total_months += int(months_only[0])

    return total_months

def clean_values(data):

    age = calculate_total_months(data[0])
    gender = [0 if data[1] == "MALE" else 1][0]

    # Check if the age is in the second line or third line
    if bool(re.search(r'\d', data[2])):
        bage = calculate_total_months(data[2])
        radiologist = data[3].split(":")[1].strip()
        print(data[2], bage)
    else:
        bage = calculate_total_months(data[3])
        radiologist = data[2].split(":")[1].strip()
        print(data[3], bage)
    print(data[0], age)

    return age, gender, bage, radiologist


age, bage, gender, radiologist= [], [], [], []
for dir in glob.glob("./data/Mex_sample_data/*"):
    for file in glob.glob(dir + "/*.txt"):
        with open(file, "r") as f:
            data = f.readlines()
            data = [line.strip() for line in data if line.strip()]

            cleaned = clean_values(data)
            age.append(cleaned[0])
            gender.append(cleaned[1])
            bage.append(cleaned[2])
            radiologist.append(cleaned[3])
mexico_data = pd.DataFrame(data=zip(age, gender, bage, radiologist), columns=['age', 'gender', 'bage', 'radiologist'])

In [ ]:
# save pandas df as csv without index
mexico_data.to_csv("./data/Mex_sample_data/mexico_data.csv", index=False)